In [ ]:
from pdf2image import convert_from_path
import pytesseract
import os
import re

# Chemin vers l'exécutable Tesseract (à modifier selon votre système)
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"

# Chemin vers poppler (pour Windows)
poppler_path ="C:/Users/21695/Downloads/Release-24.08.0-0/poppler-24.08.0/Library/bin"

def extract_text_from_pdf(pdf_path):
    """
    Extrait le texte d'un fichier PDF en utilisant l'OCR.
    """
    try:
        # Convertir le PDF en images avec poppler
        pages = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)

        # Initialiser une variable pour stocker tout le texte extrait
        full_text = ""

        # Parcourir chaque page du PDF
        for i, page in enumerate(pages):
            # Extraire le texte de l'image avec Tesseract
            text = pytesseract.image_to_string(page, lang='fra')  # Utilisez 'fra' pour le français
            full_text += f"----- Page {i + 1} -----\n{text}\n"

        return full_text
    except Exception as e:
        print(f"Erreur lors de la conversion du PDF : {e}")
        return None


def parse_pay_slip_data(text):
    """
    Analyse le texte extrait pour extraire les informations pertinentes.
    """
    data = {}

    # Rechercher les champs spécifiques avec des expressions régulières
    data["Nom"] = re.search(r"Nom\s*:\s*(\w+)", text).group(1) if re.search(r"Nom\s*:\s*(\w+)", text) else "N/A"
    data["Prénom"] = re.search(r"Prénom\s*:\s*(\w+)", text).group(1) if re.search(r"Prénom\s*:\s*(\w+)", text) else "N/A"
    data["Salaire de base"] = re.search(r"Salaire de base.*?(\d+\.\d+)", text, re.DOTALL).group(1) if re.search(r"Salaire de base.*?(\d+\.\d+)", text, re.DOTALL) else "N/A"
    data["Net à payer"] = re.search(r"NET A PAYER.*?(\d+\.\d+)", text, re.DOTALL).group(1) if re.search(r"NET A PAYER.*?(\d+\.\d+)", text, re.DOTALL) else "N/A"
    data["Usine"] = re.search(r"usine\s*:\s*(.+)", text, re.IGNORECASE).group(1) if re.search(r"usine\s*:\s*(.+)", text, re.IGNORECASE) else "N/A"

    # Extraction de la date de naissance (format AAAA-MM-JJ)
    date_naissance_match = re.search(r"(?:Date de naissance|date_naissance)\s*:\s*(\d{4}-\d{2}-\d{2})", text, re.IGNORECASE)
    data["Date de naissance"] = date_naissance_match.group(1) if date_naissance_match else "N/A"

    # Extraction du numéro de Sécurité Sociale
    data["Numéro de Sécurité Sociale"] = re.search(r"Numéro de Sécurité Sociale\s*:\s*(\d+)", text, re.IGNORECASE).group(1) if re.search(r"Numéro de Sécurité Sociale\s*:\s*(\d+)", text, re.IGNORECASE) else "N/A"

    return data


def save_to_csv(data_list, output_file="pay_slips.csv"):
    """
    Enregistre les données extraites dans un fichier CSV.
    """
    import csv

    # Écrire les données dans un fichier CSV
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data_list[0].keys())
        writer.writeheader()
        writer.writerows(data_list)


# Répertoire contenant les fiches de paie PDF
pdf_directory = "D:/Employees"  # Modifiez ce chemin selon votre dossier

# Liste pour stocker toutes les données extraites
all_data = []

# Traiter chaque fichier PDF dans le répertoire
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        print(f"Extraction des données depuis {filename}...")

        # Extraire le texte du PDF
        text = extract_text_from_pdf(pdf_path)

        if text:
            # Afficher le texte brut pour le débogage
            print("Texte extrait par OCR :")
            print(text)

            # Analyser les données
            data = parse_pay_slip_data(text)
            data["Fichier"] = filename  # Ajouter le nom du fichier pour référence
            all_data.append(data)

# Enregistrer les données dans un fichier CSV
if all_data:
    save_to_csv(all_data)
    print("Données extraites et enregistrées dans pay_slips.csv")
else:
    print("Aucune donnée extraite.")

Extraction des données depuis bulletin_de_paie_1.pdf...
Texte extrait par OCR :
----- Page 1 -----
BULLETIN DE PAIE

Informations de l'employé
Nom: Ben Ali
Prénom: Nour
Adresse: 33 Rue du Commerce
Ville: Gabès
usine: Atelier de conditionnement de Sousse
Numéro de Sécurité Sociale: 1497687807
date _naissance: 1976-01-10
Détails du salaire

Libellé Nbre de jours | Base (MG) Gain/Retenue
Salaire de base 30.5 23.58 1318.26
Congés payés 3.49 71.90
Indemnité hébergement 30.5 5 106.45
Maladie 1318.26 -61.47
CSG déductible 1318.26 -64.61
NET A PAYER 1120.52

Page 1

AaroDelices

——


Extraction des données depuis bulletin_de_paie_100.pdf...
Texte extrait par OCR :
----- Page 1 -----
BULLETIN DE PAIE

Informations de l'employé
Nom: Guerfali
Prénom: Anis
Adresse: 52 Rue des Oliviers
Ville: Sfax
usine: Usine Centrale de Sfax
Numéro de Sécurité Sociale: 8642082625
date _naissance: 1992-09-09
Détails du salaire

Libellé Nbre de jours | Base (MG) Gain/Retenue
Salaire de base 30.5 23.58 1719.74
Congé